# Cell 1: 테스트 환경 설정 및 더미 데이터 생성
먼저 테스트에 사용할 가상의 동네 데이터(서울 시청 인근 및 강남 등)를 CSV로 만듭니다.

In [1]:
import csv
import os

# 1. 테스트용 더미 데이터 생성
# (서울 시청 주변 동네들과 멀리 떨어진 강남역, 판교역 데이터)
sample_data = [
    {'dnid': '1114010300', 'dnname': '서울특별시 중구 태평로1가', 'dnlatitude': 37.5665, 'dnlongitude': 126.9779, 'dnradius': 0.3},
    {'dnid': '1114010400', 'dnname': '서울특별시 중구 을지로1가', 'dnlatitude': 37.5660, 'dnlongitude': 126.9810, 'dnradius': 0.25},
    {'dnid': '1114010500', 'dnname': '서울특별시 중구 소공동', 'dnlatitude': 37.5635, 'dnlongitude': 126.9795, 'dnradius': 0.4},
    {'dnid': '1114010200', 'dnname': '서울특별시 중구 무교동', 'dnlatitude': 37.5675, 'dnlongitude': 126.9790, 'dnradius': 0.2},
    {'dnid': '1168010100', 'dnname': '서울특별시 강남구 역삼동', 'dnlatitude': 37.4954, 'dnlongitude': 127.0333, 'dnradius': 1.5},
    {'dnid': '4113555000', 'dnname': '경기도 성남시 분당구 판교동', 'dnlatitude': 37.3929, 'dnlongitude': 127.0980, 'dnradius': 1.2},
]

csv_filename = 'test_korea_bjd.csv'

# CSV 파일 쓰기
with open(csv_filename, 'w', newline='', encoding='utf-8-sig') as f:
    writer = csv.DictWriter(f, fieldnames=['dnid', 'dnname', 'dnlatitude', 'dnlongitude', 'dnradius'])
    writer.writeheader()
    writer.writerows(sample_data)

print(f"테스트 데이터 생성 완료: {csv_filename}")

테스트 데이터 생성 완료: test_korea_bjd.csv


# Cell 2: 엔진 로드 및 초기화
만들어진 CSV 파일을 이용해 DongnaeEngine을 초기화합니다.

In [2]:
# 패키지가 설치되어 있지 않다면 현재 경로에서 import
try:
    from dongnae import DongnaeEngine
except ImportError:
    # 로컬 파일에서 직접 테스트하는 경우 (경로에 맞게 수정하세요)
    import sys
    sys.path.append('./py-core') # dongnae 패키지 경로 추가
    from dongnae.engine import DongnaeEngine

# 엔진 초기화 (자동으로 인코딩 감지 및 계수 보정 수행)
engine = DongnaeEngine(csv_filename)

print(f"엔진 초기화 완료. 총 {len(engine._dongnaes)}개의 동네가 로드되었습니다.")
print(f"   - 위도 계수: {engine._lat_coef}")
print(f"   - 경도 계수: {engine._lon_coef}")

엔진 초기화 완료. 총 6개의 동네가 로드되었습니다.
   - 위도 계수: 111.0
   - 경도 계수: 88.09


# Cell 3: [Reverse Geocoding] 내 위치는 어디? (where)
서울 시청 앞 광장 좌표(37.5663, 126.9779)를 입력해 봅니다.

In [3]:
# 서울 시청 앞 좌표
my_lat, my_lon = 37.5663, 126.9779

result = engine.where(my_lat, my_lon)

print("가장 가까운 동네 (where):")
if result:
    print(f"  - 이름: {result['dnname']}")
    print(f"  - 거리: {result['distance']} km (음수면 동네 안쪽)")
else:
    print("  - 결과 없음")

가장 가까운 동네 (where):
  - 이름: 서울특별시 중구 태평로1가
  - 거리: -0.2778 km (음수면 동네 안쪽)


# Cell 4: [Nearest Neighbor] 주변 동네 찾기 (nearest)
주변 3개의 가까운 동네를 찾아봅니다. (강남, 판교는 멀어서 안 나와야 정상)

In [4]:
# 주변 3개 동네 탐색
neighbors = engine.nearest(my_lat, my_lon, k=3)

print(f"주변 동네 Top {len(neighbors)}:")
for i, dn in enumerate(neighbors, 1):
    print(f"  {i}. {dn['dnname']} (보정 거리: {dn['distance']} km)")

주변 동네 Top 3:
  1. 서울특별시 중구 태평로1가 (보정 거리: -0.2778 km)
  2. 서울특별시 중구 소공동 (보정 거리: -0.0587 km)
  3. 서울특별시 중구 무교동 (보정 거리: -0.0353 km)


# Cell 5: [Radius Search] 반경 검색 (within)
반경 1km 이내의 동네만 검색합니다. (강남, 판교 제외 확인)

In [5]:
radius = 1.0 # 1km
nearby = engine.within(my_lat, my_lon, radius_km=radius)

print(f"반경 {radius}km 이내 동네 ({len(nearby)}개):")
for dn in nearby:
    print(f"  - {dn['dnname']} ({dn['distance']} km)")

반경 1.0km 이내 동네 (4개):
  - 서울특별시 중구 태평로1가 (-0.2778 km)
  - 서울특별시 중구 소공동 (-0.0587 km)
  - 서울특별시 중구 무교동 (-0.0353 km)
  - 서울특별시 중구 을지로1가 (0.0251 km)


# Cell 6: [Soft Geofencing] 영역 판정 (resolve)
threshold를 조절하며 내 위치가 판정 범위에 들어오는지 확인합니다.

In [6]:
# 소공동 중심 좌표 (약간 떨어져 있음)
target_lat, target_lon = 37.5635, 126.9795

print("🎯 영역 포함 여부 테스트 (Soft Geofencing):")

# 1. Strict Mode (0.8) - 엄격하게 중심부만
strict = engine.resolve(target_lat, target_lon, threshold=0.8)
print(f"  - Strict(0.8): {'포함됨' if strict else '미포함'}")

# 2. Loose Mode (1.2) - 넉넉하게 경계 포함
loose = engine.resolve(target_lat, target_lon, threshold=1.2)
print(f"  - Loose(1.2): {'포함됨' if loose else '미포함'}")

if loose:
    print(f"    -> 매칭된 곳: {loose[0]['dnname']} (Score: {loose[0]['score']})")

🎯 영역 포함 여부 테스트 (Soft Geofencing):
  - Strict(0.8): 포함됨
  - Loose(1.2): 포함됨
    -> 매칭된 곳: 서울특별시 중구 소공동 (Score: 0.0)


# Cell 7: [Text Search] 텍스트 검색 (search)
"중구 무교동" 또는 "판교"로 검색해 봅니다.

In [10]:
keywords = ["중구 무교동", "판교"]

print("텍스트 검색 결과:")
for kw in keywords:
    results = engine.search(kw)
    print(f"  Key: '{kw}'")
    for r in results:
        print(f"    - {r['dnname']} (Score: {r['score']})")

텍스트 검색 결과:
  Key: '중구 무교동'
    - 서울특별시 중구 무교동 (Score: 2)
    - 서울특별시 중구 소공동 (Score: 1)
    - 서울특별시 중구 태평로1가 (Score: 1)
    - 서울특별시 중구 을지로1가 (Score: 1)
  Key: '판교'
    - 경기도 성남시 분당구 판교동 (Score: 1)


# Cell 8: [ID Lookup] 코드 조회 (get)
법정동 코드(ID)로 직접 조회합니다.

In [8]:
target_id = '1168010100' # 역삼동

dn = engine.get(target_id)

print("ID 조회 결과:")
if dn:
    print(f"  - {dn['dnname']} (ID: {dn['dnid']})")
    print(f"  - 좌표: {dn['dnlatitude']}, {dn['dnlongitude']}")
else:
    print("  - 찾을 수 없음")

ID 조회 결과:
  - 서울특별시 강남구 역삼동 (ID: 1168010100)
  - 좌표: 37.4954, 127.0333


# Cell 9: 테스트 파일 정리
생성했던 더미 CSV 파일을 삭제합니다.

In [9]:
# 테스트 종료 후 파일 삭제
if os.path.exists(csv_filename):
    os.remove(csv_filename)
    print(f"{csv_filename} 삭제 완료.")

test_korea_bjd.csv 삭제 완료.
